In [5]:
from multiprocessing import Pool 
import csv 
from datetime import datetime 
import os 
import requests 
import pandas as pd 
import selenium 
from selenium.webdriver import Chrome 
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import NoSuchElementException, WebDriverException 
from user_agent import generate_user_agent 
from time import sleep 
#from text_processing import change_date 
#from text_processing import lemmatizator 
import numpy as np 
from os import getcwd

In [42]:
import urllib 
from urllib.parse import unquote

In [29]:
browser = Chrome(executable_path = '/home/twsea/Documents/python-mail/PrPythonAtom/homeworks/project/chromedriver')
browser.get('https://www.kinopoisk.ru/')
search_form = browser.find_element_by_class_name('header-fresh-search-partial-component__field')
search_form.send_keys('Елки')
search_form.send_keys(Keys.ENTER)
search_form = browser.find_element_by_xpath('//*[@id="block_left_pad"]/div/div[2]/div/div[2]/p/a') #клик по первому фильму
search_form.click()
browser.close()


In [47]:
string = '%E5%EB%EA%E8'

In [58]:
import urllib.parse

s = 'елки'
urllib.parse.quote(s.encode('utf-8'))

'%D0%B5%D0%BB%D0%BA%D0%B8'

In [59]:
!pip install requests bs4 user_agent selenium

In [202]:
import requests
from bs4 import BeautifulSoup
import json
import re

In [227]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package wordnet to /home/twsea/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/twsea/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [203]:
#рассматриваем две ситуации:
#либо такого фильма не существует, тогда вылетает класс document.getElementsByClassName('textgreybig')
#либо фильм существует

In [205]:
name = 'елки'
url = 'https://www.kinopoisk.ru/index.php?kp_query=' + name +'&what='
r = requests.get(url)
soup = BeautifulSoup(r.text, "html.parser")
if (soup.findAll("p", attrs={"class": ["name"]})): #если фильм существует
    href = soup.findAll("p", attrs={"class": ["name"]})[0].find('a')['href'] #ссылка на искомый фильм
    title = soup.findAll("p", attrs={"class": ["name"]})[0].find('a').text #название фильма
    href = href[:len(href)-5] #обрезаю ссылку
    #https://www.kinopoisk.ru/film/yolki-2010-493768/ord/rating/#list
    #print(href)
    new_url = 'https://www.kinopoisk.ru' + href + 'ord/rating/#list' #ссылка на отсортированные комменты (кнопка "Все")
    #print(new_url)
    #print(title)
    r = requests.get(new_url)
    soup = BeautifulSoup(r.text, "html.parser")
    comments = []
    for i in range(len(soup.findAll("div", attrs={"itemprop": ["reviews"]}))):
        comments.append(soup.findAll("div", attrs={"itemprop": ["reviews"]})[i].findAll("span", attrs={"class": ["_reachbanner_"]})[0].text)
        comments[i] = comments[i].replace('\n', ' ').replace('\r', '').replace('\t', '').replace(u'\xa0', u' ').replace('\x97', '—').replace('\x85', '...').replace('\x88', '^')
else:
    print('Введеное название фильма не найдено.')

In [228]:
def parsing_reviews (film_name): #возвращает массив типа [название, отзывы, оценка]
    url = 'https://www.kinopoisk.ru/index.php?kp_query=' + film_name +'&what='
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    if (soup.findAll("p", attrs={"class": ["name"]})): #если фильм существует
        href = soup.findAll("p", attrs={"class": ["name"]})[0].find('a')['href'] #ссылка на искомый фильм
        title = soup.findAll("p", attrs={"class": ["name"]})[0].find('a').text #название фильма
        href = href[:len(href)-5] #обрезаю ссылку
        new_url = 'https://www.kinopoisk.ru' + href + 'ord/rating/#list' #ссылка на отсортированные комменты (кнопка "Все")
        r = requests.get(new_url)
        soup = BeautifulSoup(r.text, "html.parser")
        comments = []
        #беру только 2 отзыва о фильме
        length = 2 if len(soup.findAll("div", attrs={"itemprop": ["reviews"]})) else len(soup.findAll("div", attrs={"itemprop": ["reviews"]}))
        for i in range(length):
            comments.append(soup.findAll("div", attrs={"itemprop": ["reviews"]})[i].findAll("span", attrs={"class": ["_reachbanner_"]})[0].text)
            comments[i] = comments[i].replace('\n', ' ').replace('\r', '').replace('\t', '').replace(u'\xa0', u' ').replace('\x97', '—').replace('\x85', '...').replace('\x88', '^')
        res = ''
        for i in comments: #запихиваю 2 отзыва в 1 строку
            res = res + i
        assessment = float(soup.findAll("span", attrs={"class": ["rating_ball"]})[0].text) 
        assessment = 0 if assessment < 6 else 1
        return [film_name, res, assessment]
    else:
        print('Введеное название фильма не найдено.') 

In [229]:
parsing_reviews('Елки')

['Елки',
 'Абсолютно ничего не ждав от данной картины, я решила не тратить свое время и пропустить просмотр «очередной русской комедии». Но вот четверг, утро, мне нужно скоротать 2 часа свободного времени. Альтернатив «Елкам» не нашлось, пришлось идти. Что же я увидела?  Во-первых, к своему удивлению фильм затянул, затянул в свою праздничную атмосферу. Ведь никогда мы не бываем столь близки к волшебству, как в Новый Год. Кстати, у большинства россиян Новый год является любимым праздником, в том числе и у меня, поэтому вся предновогодняя суета отражается в моем сердце яркими огоньками! И было чертовски приятно смотреть на происходящее на экране и осознавать, что любимый праздник уже через две недели!  Во-вторых, я с радостью обнаружила, что фильм не пародия, не римейк, не «оригинальный фильм, просто слизанный с ранее вышедшей зарубежной картины». В общем, в плане сценария провисаний не заметила. Может быть они и были, но, вы знаете, как-то не хочется их искать.  В-третьих, понравилось, 

In [ ]:
#нужны рейтинги измененные
#нужно напарсить данные и разбить их на обучение и трейн.
#как плохие фильмы брать, так и хорошие

In [207]:
film_name = "елки"
url = 'https://www.kinopoisk.ru/index.php?kp_query=' + film_name +'&what='
r = requests.get(url)
soup = BeautifulSoup(r.text, "html.parser")
if (soup.findAll("p", attrs={"class": ["name"]})): #если фильм существует
    href = soup.findAll("p", attrs={"class": ["name"]})[0].find('a')['href'] #ссылка на искомый фильм
    title = soup.findAll("p", attrs={"class": ["name"]})[0].find('a').text #название фильма
    href = href[:len(href)-5] #обрезаю ссылку
    new_url = 'https://www.kinopoisk.ru' + href + 'ord/rating/#list' #ссылка на отсортированные комменты (кнопка "Все")
    r = requests.get(new_url)
    soup = BeautifulSoup(r.text, "html.parser")
else:
    print('Введеное название фильма не найдено.') 

In [215]:
float(soup.findAll("span", attrs={"class": ["rating_ball"]})[0].text)

6.877

In [ ]:
url = 'https://www.kinopoisk.ru/index.php?kp_query=' + film_name +'&what='
r = requests.get(url)
soup = BeautifulSoup(r.text, "html.parser")

In [ ]:
#отдельная штучка даа..

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import nltk
#nltk.download('wordnet')
#nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [2]:
sample = np.array([[1,1,1,1]])

In [3]:
data = pd.DataFrame(sample, columns = ['name', 'review', 'raiting','quality'])

In [4]:
def parsing_reviews (film_name): #возвращает массив типа [название, отзывы, оценка]
    film_name_new = film_name.replace(' ','+')
    url = 'https://www.imdb.com/find?ref_=nv_sr_fn&q='+ film_name_new + '&s=all'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    flag = 0
    if (not soup.findAll("div", attrs={"class": ["findNoResults"]})): #если фильм существует
        if(soup.findAll("h3", attrs={"class": ["findSectionHeader"]})):
            if (soup.findAll("h3", attrs={"class": ["findSectionHeader"]})[0].text == 'Titles'):
                flag = 1

    else:
        print('Введеное название фильма не найдено.') 
        return 0 #TODO проработать возврат
    if flag == 1:
        href = soup.findAll("td", attrs={"class": ["result_text"]})[0].find('a')['href'] #ссылка на искомый фильм
        title = soup.findAll("td", attrs={"class": ["result_text"]})[0].find('a').text #название фильма
        #взять рейтинг на главной странице
        url_main = 'https://www.imdb.com' + href
        r = requests.get(url_main)
        soup = BeautifulSoup(r.text, "html.parser")
        if not soup.findAll("span", attrs={"itemprop": ["ratingValue"]}):
            return 0
        rating = float(soup.findAll("span", attrs={"itemprop": ["ratingValue"]})[0].text) 
        assessment = 0 if rating < 7.5 else 1

        #получить комментарии
        position_of_question = href.find('?')
        new_href = href[:position_of_question]
        url_reviews = 'https://www.imdb.com' + new_href + 'reviews?spoiler=hide&sort=helpfulnessScore&dir=desc&ratingFilter=0' #ссылка на reviews
        r = requests.get(url_reviews)
        soup = BeautifulSoup(r.text, "html.parser")      
        reviews = []

        #брать все отзывы, но обрезать. 5 отзывов по 20 слов

        length = 5 if len(soup.findAll("div", attrs={"class": ["content"]})) > 5 else len(soup.findAll("div", attrs={"class": ["content"]}))
        
        for i in range(length):
            review = soup.findAll("div", attrs={"class": ["content"]})[i].find('div').text
            review = review.replace('\n', ' ').replace('\r', '').replace('\t', '').replace(u'\xa0', u' ').replace('\x97', '—').replace('\x85', '...').replace('\x88', '^').replace("\'", "'")
            index = 0
            review = review[::-1]
            if len(review) > 60:             
                for i in range(60):
                    index = review.find(' ', index+1)
                review = review[:index] #обрезаем по 60 слов
            review = review[::-1]
            reviews.append(review)
        
        res = ''
        for i in reviews: #запихиваю все отзывы в 1 строку
            res = res + i + ' '
        return [[film_name, res,rating, assessment]]
    if 1 == 1:
        return 0

In [5]:
def title_of_film(film_name):
    
    film_name = film_name.replace(' ','+')
    url = 'https://www.imdb.com/find?ref_=nv_sr_fn&q='+ film_name + '&s=all'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    flag = 0
    if (not soup.findAll("div", attrs={"class": ["findNoResults"]})): #если фильм существует
        if(soup.findAll("h3", attrs={"class": ["findSectionHeader"]})):
            if (soup.findAll("h3", attrs={"class": ["findSectionHeader"]})[0].text == 'Titles'):
                flag = 1
    if flag == 1:
        title = soup.findAll("td", attrs={"class": ["result_text"]})[0].find('a').text
        return title
    else:
        return ''

In [6]:
#от 0 до 100 +-
#films = list(set(["Finding Nemo", "Finding Nemo", "The Incredibles", "Iron Man 3", "Pirates of the Caribbean: Dead Man's Chest", "Spirited Away", "Mission: Impossible - Rogue Nation", "It's Christmas, Eve", "Fantastic Beasts: The Crimes of Grindelwald", "The Predator", "Aan Devathai", "Rambo: First Blood Part II", "Sicario: Day of the Soldado", "The Hobbit: An Unexpected Journey", "The Lord of the Rings: The Fellowship of the Ring", "Replicas", "Jurassic World: Fallen Kingdom", "Thor: Ragnarok", "Django Unchained", "Harry Potter and the Deathly Hallows: Part 1", "Pirates of the Caribbean: Dead Man's Chest", "Pirates of the Caribbean: Dead Men Tell No Tales", "Hotel Transylvania 3: Summer Vacation", "Pariyerum Perumal", "Death Race 4: Beyond Anarchy", "Hunter Killer", "Maquia: When the Promised Flower Blooms", "The Hunger Games: Mockingjay - Part 1", "The Equalizer 2", "Pirates of the Caribbean: Dead Men Tell No Tales", "Rampage", "The Dark Knight Rises", "Harry Potter and the Half-Blood Prince", "Harry Potter and the Order of the Phoenix", "Mission: Impossible - Fallout", "First Man", "Avengers: Infinity War", "Maquia: When the Promised Flower Blooms", "Kingsman: The Secret Service", "The Meg", "The Greatest Showman", "Hunter Killer", "Deadpool 2", "The Holiday Calendar", "Pirates of the Caribbean: The Curse of the Black Pearl", "The Shape of Water", "The Spy Who Dumped Me", "Coco", "Mission: Impossible - Fallout", "BlacKkKlansman", "The Matrix", "Hostiles", "Kamen Rider Amazons The Movie: The Final Judgement", "The Avengers", "Now You See Me 2", "Thor: The Dark World", "River Runs Red", "The Lord of the Rings: The Two Towers", "Pulp Fiction", "Searching", "Smallfoot", "Mad Max: Fury Road", "The Equalizer 2", "Rambo: First Blood Part II", "Venom", "Upgrade", "Deadpool", "The Nutcracker and the Four Realms", "Avengers: Age of Ultron", "The Kissing Booth", "Crazy Rich Asians", "The Hobbit: The Battle of the Five Armies", "Doctor Strange", "Batman v Superman: Dawn of Justice", "Mandy", "Fantastic Beasts and Where to Find Them", "One Flew Over the Cuckoo's Nest", "Pulp Fiction", "X-Men: Days of Future Past", "A Quiet Place", "Black Panther", "Big Hero 6", "Guardians of the Galaxy Vol. 2", "Maquia: When the Promised Flower Blooms", "The Hate U Give", "Alpha", "Pacific Rim: Uprising", "Solo: A Star Wars Story", "Forrest Gump", "Hunter Killer", "To All the Boys I've Loved Before", "Avengers: Infinity War", "Dawn of the Planet of the Apes", "The Scorpion King: Book of Souls", "Now You See Me 2", "Mad Max: Fury Road", "Mamma Mia! Here We Go Again", "Rampage", "Guardians of the Galaxy Vol. 2", "Hereditary", "Fantastic Beasts: The Crimes of Grindelwald", "The Kissing Booth", "First Man", "Whiplash", "The Wolf of Wall Street", "Bad Times at the El Royale", "Avengers: Age of Ultron", "Deadpool", "In a Relationship", "Doctor Strange", "The Kissing Booth", "Rambo III", "Three Billboards Outside Ebbing, Missouri", "The Holiday Calendar", "The Equalizer", "Goosebumps 2: Haunted Halloween", "Fifty Shades Freed", "Forrest Gump", "Ocean's Eleven", "The Hobbit: An Unexpected Journey", "Twilight", "Den of Thieves", "The Greatest Showman", "The Dark Knight", "Widows", "Zoe", "Hostiles", "Hotel Transylvania 3: Summer Vacation", "The Hunger Games: Mockingjay - Part 1", "The Predator", "Mamma Mia! Here We Go Again", "Aan Devathai", "One Flew Over the Cuckoo's Nest", "Wonder Woman", "The 12th Man", "Harry Potter and the Half-Blood Prince", "Fury", "Deadpool 2", "Game Night", "Mamma Mia! Here We Go Again", "Harry Potter and the Half-Blood Prince", "The Girl in the Spider's Web", "Inglourious Basterds", "Terminator Genisys", "Harry Potter and the Chamber of Secrets", "The Lord of the Rings: The Two Towers", "Three Billboards Outside Ebbing, Missouri", "Now You See Me 2", "Venom", "BlacKkKlansman", "The Maze Runner", "Blade Runner 2049", "Hereditary", "A Quiet Place", "The Wolf of Wall Street", "The Scorpion King: Book of Souls", "Jurassic World", "Harry Potter and the Deathly Hallows: Part 1", "Rambo III", "Den of Thieves", "Spider-Man: Homecoming", "Kamen Rider Amazons The Movie: The Final Judgement", "Johnny English Strikes Again", "Mandy", "Widows", "Ralph Breaks the Internet", "River Runs Red", "Thor: The Dark World", "Maze Runner: The Death Cure", "The Purge: Election Year", "Finding Nemo", "Maquia: When the Promised Flower Blooms", "Iron Man 3", "The Kissing Booth", "Split", "Jurassic World: Fallen Kingdom", "Night School", "Alien: Covenant", "Mission: Impossible - Rogue Nation", "Fury", "Fantastic Beasts and Where to Find Them", "The Other Side of the Wind", "The Maze Runner", "BlacKkKlansman", "Thor: The Dark World", "Thor: The Dark World", "Pirates of the Caribbean: Dead Men Tell No Tales", "Death Race 4: Beyond Anarchy", "Now You See Me 2", "The Scorpion King: Book of Souls", "The Spy Who Dumped Me", "Inglourious Basterds", "Harry Potter and the Deathly Hallows: Part 2", "Hotel Artemis", "Harry Potter and the Prisoner of Azkaban", "Thor: The Dark World", "Pulp Fiction", "Red Sparrow", "The Nutcracker and the Four Realms", "Venom", "The Grinch", "X-Men: Days of Future Past", "Adrift", "The Miseducation of Cameron Post", "Spirited Away", "The Predator", "Fantastic Beasts: The Crimes of Grindelwald", "In a Relationship", "Widows", "Inglourious Basterds", "Rambo III", "The 12th Man", "The Nutcracker and the Four Realms", "The Equalizer", "Doctor Strange", "Rampage", "Night at the Museum: Secret of the Tomb", "Zoe", "The Holiday Calendar", "Aan Devathai", "Pariyerum Perumal", "Hotel Transylvania 2", "Red Sparrow", "The Other Side of the Wind", "The Lord of the Rings: The Two Towers", "The Shape of Water", "Siberia", "Maquia: When the Promised Flower Blooms", "A Star Is Born", "Fifty Shades Freed", "Widows", "Night School", "Solo: A Star Wars Story", "The Grinch", "Upgrade", "Ant-Man and the Wasp", "Bird Box", "Mad Max: Fury Road", "Avengers: Infinity War", "Fifty Shades Freed", "The Hunger Games: Mockingjay - Part 1", "The Nutcracker and the Four Realms", "First Man", "Death Race 4: Beyond Anarchy", "Spirited Away", "Kamen Rider Amazons The Movie: The Final Judgement", "The Other Side of the Wind", "Pirates of the Caribbean: Dead Men Tell No Tales", "Bohemian Rhapsody", "Whiplash", "First Man", "Kingsman: The Secret Service", "Kin", "The Lord of the Rings: The Two Towers", "Mamma Mia! Here We Go Again", "Siberia", "Interstellar", "The Predator", "The Nutcracker and the Four Realms", "Female War: A Nasty Deal", "Fight Club", "Ocean's Eight", "Searching", "The Hate U Give", "Fifty Shades Freed", "Ender's Game", "Spider-Man: Homecoming", "A Quiet Place", "Terminal", "Papillon", "Harry Potter and the Deathly Hallows: Part 2", "First Man", "Mission: Impossible - Rogue Nation", "The Meg", "A-X-L", "Guardians of the Galaxy", "The Nun", "Hotel Artemis", "Dawn of the Planet of the Apes", "A Quiet Place", "Ocean's Eight", "Annihilation", "Pirates of the Caribbean: On Stranger Tides", "Christopher Robin", "The Scorpion King: Book of Souls", "Time Freak", "Crazy Rich Asians", "Widows", "Bird Box", "Ocean's Eleven", "BlacKkKlansman", "Bird Box", "Kamen Rider Amazons The Movie: The Final Judgement", "The Girl in the Spider's Web", "Solo: A Star Wars Story", "Skyscraper", "Deadpool 2", "Ant-Man and the Wasp", "River Runs Red", "Spectre", "Tomb Raider","Sicario: Day of the Soldado", "Wonder Woman", "In a Relationship", "The Lord of the Rings: The Two Towers", "Harry Potter and the Prisoner of Azkaban", "Pirates of the Caribbean: Dead Man's Chest", "Mandy", "River Runs Red", "The Wolf of Wall Street", "Mile 22", "Johnny English Strikes Again", "The Purge", "The Hate U Give", "Fifty Shades Freed", "Kamen Rider Amazons The Movie: The Final Judgement", "Widows", "The Miseducation of Cameron Post", "X-Men: Apocalypse", "To All the Boys I've Loved Before", "Terminal", "Pariyerum Perumal", "The Nutcracker and the Four Realms", "Dawn of the Planet of the Apes", "The Dark Knight", "Spectre", "Harry Potter and the Chamber of Secrets", "Mission: Impossible - Ghost Protocol", "Logan", "Inglourious Basterds", "Batman Begins", "Gone Girl", "Upgrade", "Overlord", "Ocean's Eight", "Fight Club", "Overboard", "Mission: Impossible", "First Blood", "Mission: Impossible - Fallout", "Ender's Game", "The Grinch", "Harry Potter and the Half-Blood Prince", "Inglourious Basterds", "Finding Nemo", "Justice League", "X-Men: Days of Future Past", "Papillon", "Den of Thieves", "Fight Club", "Overlord", "The Hobbit: The Battle of the Five Armies", "Alien: Covenant", "Hotel Transylvania 3: Summer Vacation", "Forrest Gump", "Batman Begins", "The Nun", "Aan Devathai", "Pirates of the Caribbean: Dead Men Tell No Tales", "Time Freak", "Hotel Transylvania 3: Summer Vacation", "Blade Runner", "Now You See Me 2", "The 12th Man", "Crazy Rich Asians", "Annihilation", "To All the Boys I've Loved Before", "Ant-Man", "The Nun", "Big Hero 6", "Thor: The Dark World", "X-Men: Days of Future Past", "Blindspotting", "The Purge: Election Year", "The Avengers", "Terminal", "The Miseducation of Cameron Post", "Avengers: Age of Ultron", "Forrest Gump", "Pirates of the Caribbean: Dead Man's Chest", "War for the Planet of the Apes", "Rambo: First Blood Part II", "First Man", "Searching", "The Holiday Calendar", "Star Wars: The Force Awakens", "Female War: A Nasty Deal", "Now You See Me 2", "Bird Box", "Doctor Strange", "Pirates of the Caribbean: On Stranger Tides", "Mission: Impossible", "Justice League", "Thor: Ragnarok", "Bohemian Rhapsody", "Incredibles 2", "Insurgent", "Kingsman: The Secret Service", "The Purge: Election Year", "The Miseducation of Cameron Post", "Spectre","The Scorpion King: Book of Souls", "Inception", "The Thinning: New World Order", "Rambo III", "Zoe", "The Predator", "Bad Times at the El Royale", "Doctor Strange", "Mission: Impossible - Rogue Nation", "Rampage", "X-Men: Apocalypse", "The Darkest Minds", "Siberia", "Ralph Breaks the Internet", "The Purge: Election Year", "Upgrade", "Annihilation", "Maze Runner: The Death Cure", "Twilight", "Ocean's Eight", "Harry Potter and the Order of the Phoenix", "Finding Nemo", "Spirited Away", "Spider-Man: Homecoming", "Smallfoot", "The Hobbit: An Unexpected Journey", "Rambo III", "The First Purge", "Logan", "The Purge: Election Year", "Blindspotting", "Skyscraper", "The Hobbit: An Unexpected Journey", "Hotel Transylvania 3: Summer Vacation", "Thor", "Inglourious Basterds", "Ocean's Eight", "Insurgent", "Christopher Robin", "The Miseducation of Cameron Post", "Ocean's Eleven", "Mandy", "Blade Runner 2049", "Pulp Fiction", "Kingsman: The Secret Service", "Alpha", "Exodus: Gods and Kings", "Mamma Mia! Here We Go Again", "The Equalizer", "X-Men: Apocalypse", "Harry Potter and the Half-Blood Prince", "Mission: Impossible", "Mission: Impossible", "Finding Nemo", "Avengers: Age of Ultron", "Fantastic Beasts and Where to Find Them", "Alien: Covenant", "First Blood", "Aan Devathai", "Upgrade", "Rambo: First Blood Part II", "Jurassic World", "Christopher Robin", "One Flew Over the Cuckoo's Nest", "The 12th Man", "The Equalizer 2", "Maquia: When the Promised Flower Blooms", "Inception", "Pirates of the Caribbean: Dead Man's Chest", "Death Race 4: Beyond Anarchy", "The Seven Deadly Sins: Prisoners of the Sky", "Twilight", "Finding Nemo", "The Nun", "Batman Begins", "River Runs Red", "The Imitation Game", "Goosebumps 2: Haunted Halloween", "Incredibles 2", "A Quiet Place", "Fantastic Beasts: The Crimes of Grindelwald", "Thor: Ragnarok", "The Predator", "Red Sparrow", "Ant-Man", "Ready Player One", "Avengers: Infinity War", "Dawn of the Planet of the Apes", "Harry Potter and the Philosopher's Stone", "The Shawshank Redemption", "Insurgent", "Bird Box", "Blindspotting", "Maze Runner: The Death Cure", "Twilight", "Forrest Gump", "Pirates of the Caribbean: Dead Man's Chest", "Terminator Genisys", "The Lord of the Rings: The Two Towers", "Exodus: Gods and Kings","Coco", "Harry Potter and the Half-Blood Prince", "The Purge: Anarchy", "Hotel Transylvania 3: Summer Vacation", "Bohemian Rhapsody", "A Star Is Born", "The Hunger Games: Mockingjay - Part 1", "Insurgent", "Thor", "Christopher Robin", "The Grinch", "X-Men: Days of Future Past", "Zoe", "Sorry to Bother You", "Ant-Man and the Wasp", "The Predator", "The Hobbit: An Unexpected Journey", "Jurassic World", "Pirates of the Caribbean: At World's End", "Overboard", "War for the Planet of the Apes", "Iron Man 3", "Hostiles", "Star Wars", "Venom", "Thor: Ragnarok", "Gone Girl", "Pariyerum Perumal", "Kingsman: The Secret Service", "Avengers: Infinity War", "Twilight", "Ender's Game", "One Flew Over the Cuckoo's Nest", "Blindspotting", "The Equalizer", "Bohemian Rhapsody", "The Spy Who Dumped Me", "Ocean's Eleven", "Hereditary", "Star Wars: The Last Jedi", "Insurgent", "Big Hero 6", "The First Purge", "Crazy Rich Asians", "Creed II", "The Matrix", "Alpha", "Rambo: First Blood Part II", "Pirates of the Caribbean: At World's End", "Interstellar", "The Hunger Games: Mockingjay - Part 1", "Adrift", "Harry Potter and the Order of the Phoenix", "Smallfoot", "Den of Thieves", "Avengers: Age of Ultron", "Thor: Ragnarok", "Harry Potter and the Order of the Phoenix", "Forrest Gump", "Hostiles", "Star Wars: The Last Jedi", "Guardians of the Galaxy Vol. 2", "River Runs Red", "Mandy", "The Wolf of Wall Street", "Inglourious Basterds", "Spectre", "Incredibles 2", "Pulp Fiction", "Harry Potter and the Prisoner of Azkaban", "Black Panther", "The Scorpion King: Book of Souls", "Den of Thieves", "Hotel Transylvania 3: Summer Vacation", "Bad Times at the El Royale", "Replicas", "The Lord of the Rings: The Fellowship of the Ring", "First Blood", "The Dark Knight Rises", "Hostiles", "Fight Club", "It's Christmas, Eve", "Pirates of the Caribbean: Dead Men Tell No Tales", "Halloween", "Halloween", "The Hobbit: The Battle of the Five Armies", "Harry Potter and the Chamber of Secrets", "It's Christmas, Eve", "It's Christmas, Eve", "Ocean's Eight", "Hostiles", "X-Men: Days of Future Past", "The Meg", "Fight Club", "The Godfather", "A-X-L", "Halloween", "Ready Player One", "Ender's Game", "Forrest Gump", 'Billy: The Early Years',"Planes, Trains and Automobiles (1987)", "The Legend of Drunken Master (1994)", "The Fighter (2010)", "Once Upon a Time in the West (1968)", "Beverly Hills Cop (1984)", "Sideways (2004)", "Double Indemnity (1944)", "The Lord of the Rings: The Two Towers (2002)", "It's a Wonderful Life (1946)", "The Aviator (2004)", "Million Dollar Baby (2004)", "The Lion King (1994)", "Lost in Translation (2003)", "The Silence of the Lambs (1991)", "The Hurt Locker (2008)", "Aladdin (1992)", "Sideways (2004)", "On the Waterfront (1954)", "Gladiator (2000)", "The Mission (1986)", "The Hurt Locker (2008)", "Shutter Island (2010)", "Batman (1989)", "The Artist (2011)", "House of Sand and Fog (2003)", "Chocolat (2000)", "Raising Arizona (1987)", "Rango (2011)", "Blue Jasmine (2013)", "Best in Show (2000)", "Down by Law (1986)", "Eastern Promises (2007)", "Incendies (2010)", "Gosford Park (2001)", "Touch of Evil (1958)", "The Impossible (2012)", "4 Months, 3 Weeks and 2 Days (2007)", "Milk (2008)", "Minority Report (2002)", "The Piano (1993)", "An Education (2009)", "The Bourne Ultimatum (2007)", "Edward Scissorhands (1990)", "Traffic (2000)", "The Wave (2008)", "Strangers on a Train (1951)", "Breakfast at Tiffany's (1961)", "Three Days of the Condor", "The Godfather: Part III (1990)", "Billy Elliot (2000)","The Exorcist (1973)", "Gosford Park (2001)", "Trainspotting (1996)", "Rounders (1998)", "Good Bye, Lenin! (2003)", "Strangers on a Train (1951)", "Following (1998)", "Hangover (2014)", "Short Cuts (1993)", "Kramer Vs. Kramer (1979)", "Sunshine (2007)", "Flight (2012)", "Vacation (2015)", "The Grapes of Wrath (1940)", "The Way Way Back (2013)", "The Grapes of Wrath (1940)", "Synecdoche, New York (2008)", "In the Heat of the Night (1967)", "The Fifth Element (1997)", "Sunshine (2007)", "Stardust (2007)", "Pulp Fiction (1994)"]))

In [7]:
data_movies = pd.read_csv('movies.dat',  sep='::', header = None)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [8]:
data_movies = data_movies.drop(columns=[0])
data_movies.columns = ['title', 'genre']

In [9]:
data_movies.head()

,title,genre
0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,Jumanji (1995),Adventure|Children|Fantasy
2,Grumpier Old Men (1995),Comedy|Romance
3,Waiting to Exhale (1995),Comedy|Drama|Romance
4,Father of the Bride Part II (1995),Comedy


In [117]:
data_movies[data_movies['title'] == "Second Chorus (1940)"]

,title,genre
8173,Second Chorus (1940),Comedy|Musical|Romance


In [ ]:
for i in data_movies['title'][8173:]:
    #if not i in list(data['name']): #проверяет повторы для data
        #if not i in [sample[i][0] for i in range(len(sample))]: #проверяет повторы для sample
            print(i)
            if (not parsing_reviews(i) == 0):
                sample = np.append(sample, parsing_reviews(i), axis=0)

Second Chorus (1940)
Class of Nuke 'Em High Part II: Subhumanoid Meltdown (1991)
Murder on the Orient Express (1974)
Mask (1985)
Alexander's Ragtime Band (1938)
Boston Strangler, The (1968)


In [45]:
#sample = sample[1:]

In [82]:
#data = pd.read_csv('film_reviews.csv',  sep='\t')
data = pd.DataFrame(sample, columns = ['name', 'review', 'raiting','quality'])

In [83]:
#data = data.drop(columns=['Unnamed: 0','Unnamed: 0.1'])
data

,name,review,raiting,quality
0,Toy Story (1995),as the favourite characters for adults.Overall...,8.3,1
1,Jumanji (1995),"being as suitable as Jaws was for children, bu...",6.9,0
2,Grumpier Old Men (1995),a special place in my heart. I was personally ...,6.6,0
3,Waiting to Exhale (1995),"unfeeling creatures. So basically, I feel that...",5.8,0
4,Father of the Bride Part II (1995),I don't think George Banks could possibly have...,6.0,0
5,Heat (1995),want to lead. Pacino's cop is less easy to sym...,8.2,1
6,Sabrina (1995),"the) Moonlight"" and ""How Can I Remember?"", are...",6.3,0
7,Tom and Huck (1995),scenes. The graveyard scene is well done. Inju...,5.6,0
8,Sudden Death (1995),"the gunplay and the action during the game), a...",5.7,0
9,GoldenEye (1995),"produced by Eon Productions, and the first to ...",7.2,0


In [84]:
#data.to_csv('film_reviews_4493_original.csv', sep='\t', index=False)

In [28]:
'''
#замена русских названий на английские
for k in films:
    title = title_of_film(k)
    print(title)
    for i in data['name']:
        if title == i:
            index = data[data['name'] == i].index[0]
            data.name[index] = k
            '''

"\n#замена русских названий на английские\nfor k in films:\n    title = title_of_film(k)\n    print(title)\n    for i in data['name']:\n        if title == i:\n            index = data[data['name'] == i].index[0]\n            data.name[index] = k\n            "

In [138]:
#обрезаю reviews
'''for p in range(161):
    index = 0
    for i in range(100):
        index = data.loc[p, 'review'].find(' ', index+1)
    data.loc[p, 'review'] = data.loc[p, 'review'][:index]
'''

In [15]:
data

,name,review,raiting,quality
0,Toy Story (1995),Andy's toys live a reasonable life of fun and ...,8.3,1
1,Jumanji (1995),The smash hit 1995 film Jumanji- based on the ...,6.9,0
2,Grumpier Old Men (1995),Some people see this as an inferior sequel to ...,6.6,0
3,Waiting to Exhale (1995),I read some of the other comments comcerning t...,5.8,0
4,Father of the Bride Part II (1995),"I enjoyed this film, as I did Father of the Br...",6.0,0
5,Heat (1995),One of the most amazing things about Heat is t...,8.2,1
6,Sabrina (1995),"I saw the original ""Sabrina"" before ever seein...",6.3,0
7,Tom and Huck (1995),We all know the story. There have been some pr...,5.6,0
8,Sudden Death (1995),"After the success of ""Die Hard"" there have bee...",5.7,0
9,GoldenEye (1995),GoldenEye (1995) is my number 1 personal favor...,7.2,0


In [16]:
data.review[5]

"One of the most amazing things about Heat is the scale of the film; it is nearly three hours long and packed to bursting with mind-blowing visuals. It seems one of Michael Mann's main priorities was to make a film with a dreamlike feel to it, to portray LA as a dusty oil-painting on which complex characters could play out their lives. One of the main themes is the similarity of the career criminal and the street-wise cop. It is fascinating to find yourself really feeling for DeNiro's tragic bank-robber, Sound like a bold statement? Devotees of classic cops and robbers flicks of old will no doubt take exception, but I believe that Michael Mann achieved some measure of perfection with Heat. To break this three-hour gem of a film down to its core, this is a film about men - strong men - and the supporting role that he women of the film have on them for better or worse. Take Pacino as good cop Vincent Hanna: one of the most intense characterizations of the tragic hero that I 'Heat,' a fil

In [18]:
s = 'I read some of the other comments comcerning this movie. And personally i thought that this movie was greatly misinterpreted. This movie (while have some "man-bashing" elements) is more about the friendship of four strong real black women who represent This was a good movie, even though I\'m not the target audience.  It\'s nice to see that black women had a movie about them, and thanks to the star power of Whitney Houston it became no.1 and was a While this movie truly is bound to be more appealing to women than men, it is a touching story of four best friends. How they stick together and grow together through life\'s mistakes, joys, and bad taste in men. I I have the video of this movie and it grew on me as time passed. It\'s not a great movie but it is enjoyable and as has been pointed out by others, it feels good to see a movie about Angela Basset was good as expected, but Whitney has no Range as an actress. The screenplay also neglected to portray, on film, the greatness of this novel.  Instead of promoting sisterhood, they emphasized the canine-qualities of men.  Read '

In [27]:
index = 0
for i in range(40000):
    index = s.find(' ', index+1)

In [28]:
index

1

In [19]:
parsing_reviews('Tom & Viv (1994)')